<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/01-Mi-segundo-modelo-Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Mi segundo modelo
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Emilio Andrade &bull; 11/29/25
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este cuaderno está diseñado practicar en el entrenamiento y evaluación de un modelo de regresión lineal utilizando datos reales. Los objetivos principales son:

- Entrenar un modelo de regresión lineal.
- Evaluar el ajuste del modelo.


Del conjunto de datos disponible [aquí](https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv), se desea predecir la variable `price` en función de las variables `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`.

In [1]:
# Paquetes necesarios — Emilio Andrade

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    RobustScaler,
    StandardScaler,
    Normalizer,
    QuantileTransformer,
    PowerTransformer
)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import files


---
## <font color='264CC7'> Lectura de Datos </font>


Realiza la lectura de los datos y presenta las primeras filas del conjunto de datos.

In [2]:
# Lectura de datos desde el selector de archivos — Emilio Andrade


# Seleccionar y subir archivo
uploaded = files.upload()

file_name = list(uploaded.keys())[0]

df = pd.read_csv(file_name)

# Mostrar primeras filas
display(df.head())


Saving kingCountyHouseData.csv to kingCountyHouseData.csv


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


---
## <font color='264CC7'> Preprocesamiento de datos </font>

Realiza un escalado de los datos, prueba diferentes técnicas y selecciona la que mejor se ajuste a los datos.

In [3]:
# Preprocesamiento de datos — Escalado

# Selección de características
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']
X = df[features]

# Escaladores
scalers = {
    'MaxAbsScaler': MaxAbsScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler(),
    'StandardScaler': StandardScaler(),
    'Normalizer': Normalizer(),
    'QuantileTransformer': QuantileTransformer(output_distribution='normal'),
    'PowerTransformer': PowerTransformer()
}

# Diccionario donde guardaremos los DataFrames escalados
scaled_data = {}

for name, scaler in scalers.items():
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    scaled_data[name] = X_scaled
    print(f"\nEscalado con {name}:")
    display(X_scaled.head())



Escalado con MaxAbsScaler:


,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.087149,0.003421,0.285714
1,0.090909,0.28125,0.189808,0.004385,0.571429
2,0.060606,0.12500,0.056869,0.006056,0.285714
3,0.121212,0.37500,0.144756,0.003028,0.285714
4,0.090909,0.25000,0.124077,0.004893,0.285714



Escalado con MinMaxScaler:


,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.067170,0.003108,0.0
1,0.090909,0.28125,0.172075,0.004072,0.4
2,0.060606,0.12500,0.036226,0.005743,0.0
3,0.121212,0.37500,0.126038,0.002714,0.0
4,0.090909,0.25000,0.104906,0.004579,0.0



Escalado con RobustScaler:


...

,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.000520,0.000173,0.204438,0.978879,0.000173
1,0.000390,0.000293,0.334440,0.942417,0.000260
2,0.000199,0.000100,0.076773,0.997049,0.000100
3,0.000745,0.000559,0.364961,0.931022,0.000186
4,0.000364,0.000242,0.203567,0.979061,0.000121



Escalado con QuantileTransformer:


,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,-0.334851,-1.321946,-1.097845,-0.506348,-5.199338
1,-0.334851,0.072829,0.699108,-0.110348,0.748009
2,-1.449682,-1.321946,-2.107953,0.580524,-5.199338
3,0.697507,1.266860,0.066541,-0.711981,-5.199338
4,-0.334851,-0.160010,-0.292676,0.136866,-5.199338



Escalado con PowerTransformer:


,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,-0.352407,-1.571853,-1.121655,-0.331682,-0.975610
1,-0.352407,0.250882,0.706456,-0.032289,1.029432
2,-1.605435,-1.571853,-2.107060,0.336051,-0.975610
3,0.722633,1.136536,0.065492,-0.484436,-0.975610
4,-0.352407,-0.072697,-0.296964,0.095276,-0.975610


### Selección del mejor método de escalado

Para este conjunto de datos, **el mejor escalado preliminar es *StandardScaler***.  
La razón es que las variables como `sqft_living`, `sqft_lot` y `bathrooms` tienen rangos y distribuciones muy diferentes, y *StandardScaler* centra todas las características en media 0 y varianza 1, lo que estabiliza el comportamiento del modelo sin distorsionar las relaciones entre variables.

Otros métodos como `Normalizer` o `QuantileTransformer` deforman excesivamente los datos, mientras que `RobustScaler` es útil para muchos outliers, pero aquí los valores quedan demasiado achatados o desplazados. Por ello, **StandardScaler ofrece la transformación más equilibrada y apropiada para regresión lineal**.


---
## <font color='264CC7'> Características y etiquetas </font>

Separa las características y las etiquetas y mostrar la dimensión de cada elemento.

In [4]:
# Características y etiquetas

# Definir características (X) y etiqueta (y)
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']
X = df[features]
y = df['price']

# Mostrar dimensiones
print("Dimensión de X:", X.shape)
print("Dimensión de y:", y.shape)


Dimensión de X: (21613, 5)
Dimensión de y: (21613,)


---
## <font color='264CC7'> Conjunto de entrenamiento y prueba </font>

Divide los datos en dos conjuntos: uno para entrenamiento y otro para prueba y visualiza la dimensión de cada conjunto. Usar `random_state` igual al últmo número de tu cédula.

Revisa la distribución de las etiquetas en los conjuntos de entrenamiento y prueba.

In [5]:
# Conjunto de entrenamiento y prueba


random_state = 7

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=random_state
)

# Dimensiones
print("Dimensión de X_train:", X_train.shape)
print("Dimensión de X_test:", X_test.shape)
print("Dimensión de y_train:", y_train.shape)
print("Dimensión de y_test:", y_test.shape)

# Distribución de las etiquetas
print("\nDistribución de y_train:")
display(y_train.describe())

print("\nDistribución de y_test:")
display(y_test.describe())



Dimensión de X_train: (17290, 5)
Dimensión de X_test: (4323, 5)
Dimensión de y_train: (17290,)
Dimensión de y_test: (4323,)

Distribución de y_train:


,price
count,1.729000e+04
mean,5.404569e+05
std,3.680163e+05
min,7.500000e+04
25%,3.200000e+05
50%,4.500000e+05
75%,6.480000e+05
max,7.700000e+06



Distribución de y_test:


,price
count,4.323000e+03
mean,5.386134e+05
std,3.635878e+05
min,8.995000e+04
25%,3.249500e+05
50%,4.500000e+05
75%,6.350000e+05
max,6.885000e+06


---
## <font color='264CC7'> Entrenamiento </font>

Define un modelo de regresión lineal y entrenalo con los datos de entrenamiento.

In [6]:
# Entrenamiento del modelo

# Crear el modelo de regresión lineal
modelo = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
modelo.fit(X_train, y_train)

# Mostrar los coeficientes y el intercepto del modelo
print("Coeficientes del modelo:", modelo.coef_)
print("Intercepto:", modelo.intercept_)


Coeficientes del modelo: [-5.80406679e+04  5.39638967e+03  3.14586331e+02 -4.51918950e-01
  1.99871192e+03]
Intercepto: 72585.90188865259


---
## <font color='264CC7'> Evaluación </font>

Realiza la predicciones con el conjunto de prueba y muestra los valores en una tabla.

In [7]:
# Predicciones del modelo

# Realizar predicciones con el conjunto de prueba
y_pred = modelo.predict(X_test)

# Crear tabla comparativa entre valores reales y predicciones
df_predicciones = pd.DataFrame({
    'Actual': y_test.values,
    'Predicción': y_pred
})

# Mostrar las primeras filas de la tabla
display(df_predicciones.head())


,Actual,Predicción
0,462000.0,182299.162853
1,768000.0,747813.365439
2,920000.0,688272.716229
3,263000.0,412810.903255
4,427500.0,620888.127055


Calcula el error cuadrático medio.

In [8]:
# Error cuadrático medio

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

print("Error cuadrático medio (MSE):", mse)


Error cuadrático medio (MSE): 66216707795.69068


Escribe el error cuadrático medio para cada tipo de escalado:

- Escalado MaxAbsScaler:
- Escalado MinMaxScaler:
- Escalado RobustScaler:
- Escalado StandardScaler:
- Escalado Normalizer:
- Escalado QuantileTransformer:
- Escalado PowerTransformer:

In [9]:
# MSE para cada tipo de escalado

resultados_mse = {}

for nombre, X_scaled in scaled_data.items():
    # División train/test
    X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
        X_scaled, y, test_size=0.2, random_state=random_state
    )

    # Entrenar modelo
    modelo_s = LinearRegression()
    modelo_s.fit(X_train_s, y_train_s)

    # Predicción
    y_pred_s = modelo_s.predict(X_test_s)

    # MSE
    mse_s = mean_squared_error(y_test_s, y_pred_s)

    # Guardar resultado
    resultados_mse[nombre] = mse_s

# Mostrar los resultados
for nombre, mse in resultados_mse.items():
    print(f"{nombre}: {mse}")


MaxAbsScaler: 66216707795.69041
MinMaxScaler: 66216707795.69041
RobustScaler: 66216707795.69041
StandardScaler: 66216707795.69041
Normalizer: 106369059168.0407
QuantileTransformer: 80092099372.67247
PowerTransformer: 80425909546.80829


Escribe un párrafo sobre qué hace cada escalado no visto en clase:

- **Escalado ...:** ...

### Explicación de los métodos de escalado no vistos en clase

- **MaxAbsScaler:** este método divide cada valor entre el máximo valor absoluto de su columna, manteniendo la forma original de la distribución. Es ideal cuando los datos ya están centrados alrededor de cero y no se quiere alterar la interpretación de las magnitudes.

- **RobustScaler:** utiliza la mediana y el rango intercuartílico (IQR) para escalar los datos. Esto hace que sea muy resistente a valores atípicos, ya que no depende de la media ni de la desviación estándar, que son sensibles a outliers.

- **Normalizer:** toma cada fila (no cada columna) y la transforma para que tenga una longitud unitaria. Es útil para problemas donde importa la dirección del vector más que su magnitud, pero no es adecuado para regresión porque distorsiona relaciones entre variables.

- **QuantileTransformer:** transforma los datos para que sigan una distribución uniforme o normal. Ordena los valores y les asigna cuantiles, por lo que elimina completamente la forma original de la distribución. Sirve cuando hay muchas colas largas o valores extremos.

- **PowerTransformer:** aplica una transformación no lineal diseñada para hacer que los datos se parezcan más a una distribución normal. Reduce asimetrías (skewness) y estabiliza la varianza, siendo útil en datos muy dispersos o con escalas muy diferentes.


